# Connect to MySQL
- get list of pngs
- samba mapped network drive

In [ ]:
from pdf_ocr import *

In [ ]:
tqdm.pandas()
creds = yaml.load(open('ssl/cred.yml'), Loader=yaml.FullLoader)

# https://towardsdatascience.com/sql-on-the-cloud-with-python-c08a30807661
config = {
    'user': 'nlpfx',
    'password': creds['password'],
    'host': creds['host'],
    'client_flags': [ClientFlag.SSL],
    'ssl_ca': 'ssl/server-ca.pem',
    'ssl_cert': 'ssl/client-cert.pem',
    'ssl_key': 'ssl/client-key.pem',
    'database': 'ocrProcessing'
}


In [ ]:
# now we establish our connection
cnxn = mysql.connector.connect(**config)
mycursor = cnxn.cursor()

# https://drive.google.com/file/d/1d7e-cUzP3fXduV6quxgaTEA09vlevdMo/view
# get headers with show columns
mycursor.execute("show columns from ocrLogs")
header = [x[0] for x in mycursor.fetchall()]

# select table
# df = pd.read_sql('SELECT * FROM table_name', con=db_connection)
mycursor.execute("""
select * from ocrLogs
where status=0 
and file_type='application/pdf'
and docName not in ('16070024921708', '16070024929618', '16070025038368')
limit 2
""")
myresult = mycursor.fetchall()

df = pd.DataFrame(myresult, columns = header)
df.tail()

In [ ]:
from collections import Counter
Counter(df['file_type']).most_common()

In [ ]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/Users/byukan/document-classification/ssl/[redacted].json"

In [ ]:
'/var/www/nlpfx.com/public/uploads/upd/1607002743'.split('/')[-1]

In [ ]:
df['full_path'] = df.apply(lambda x: '/Users/byukan/upds/' + x.docLocation.split('/')[-1]+'/' + x.docName, axis=1)

In [ ]:
df['full_path'][0]

In [ ]:
# get the full path
# /var/www/nlpfx.com/public/uploads/upd

# docLocation = '1606977200'
# docOriginalName = 'adafruit_national_hispanic_heritage_month_2020_blog-3-356x267.jpg'
# full_path = f'/var/www/nlpfx.com/public/uploads/upd/{docLocation}/{docOriginalName}'
# full_path

In [ ]:
# text file with all the names /var/www/nlpfx.com/uploads/

## OCR

In [ ]:
# pytesseract.image_to_string(Image.open(full_path))

In [ ]:
# '/var/www/nlpfx.com/public/uploads/upd/1607002738/16070027386985'
# pytesseract.image_to_string(Image.open('/var/www/nlpfx.com/public/uploads/upd/1607002738/16070027386985'
# ))

In [ ]:
# # https://cloud.google.com/storage/docs/downloading-objects#storage-download-object-python
# from google.cloud import storage

# client = storage.Client()
# bucket = client.bucket('upds')

# # blobs = list(bucket.list_blobs(prefix=)

In [ ]:
# blob = bucket.blob('1607002492/16070024921708')

In [ ]:
# %timeit blob.download_as_string();
# %timeit blob.download_to_file();
# %timeit blob.download_to_filename();
# %timeit blob.download_as_bytes();

In [ ]:
# p = 'gs://upds/1607002492/16070024921708'
# pytesseract.image_to_string(Image.open(p))

In [ ]:
# def ocr(x):
#     try: return pytesseract.image_to_string(Image.open(x))
#     except: return ""

# df['ocr_text'] = df['full_path'].apply(ocr)

### pdf and docx
- PyPDF2 does not ocr images in pdfs, hence we have to convert pdfs to pngs first

In [ ]:
# !pip install PyPDF2
import PyPDF2
# pdf_path = r"data/opthalmologist notes - Yukan, Brant_DOB_6_12_1992_DOS_7_28_2020_TO _DOS_5_8_2020.pdf"

pdf_path = '/Users/byukan/upds/1607002743/16070027436638'
filename = pdf_path.split('/')[-1]


# pdfFileObject = open(pdf_path, 'rb')
# pdfReader = PyPDF2.PdfFileReader(pdfFileObject)
# '\n'.join([pdfReader.getPage(i).extractText() for i in range(pdfReader.numPages)])[:100]

In [ ]:
df['ocr_text'] = df['full_path'].progress_apply(pdf_ocr)

In [ ]:
df['num_pages'] = df['ocr_text'].apply(lambda x: x[0])
df['ocr_text'] = df['ocr_text'].apply(lambda x: x[1])

In [ ]:
df

In [ ]:
# # !pip install python-docx
# import docx
# doc = docx.Document('data/DreamSystem_BYukan.docx')
# '\n'.join([x.text for x in doc.paragraphs])[:100]

# Extract name

In [ ]:
# from spacy.cli import download
# download('en')
# download('en_core_web_md')

In [ ]:
os.path.basename(df['full_path'][1])

In [ ]:
s = pytesseract.image_to_string(Image.open('/Users/byukan/document-classification/data/images_opthalmologist notes - Yukan, Brant_DOB_6_12_1992_DOS_7_28_2020_TO _DOS_5_8_2020/opthalmologist notes - Yukan, Brant_DOB_6_12_1992_DOS_7_28_2020_TO _DOS_5_8_2020_0.png'))
extract_name(s)

In [ ]:
df['extracted_names'] = df['ocr_text'].apply(lambda x: extract_name(x))

In [ ]:
df['best_guess_name'] = df['extracted_names'].apply(lambda x: x[0][0] if x else '')

In [ ]:
pdf_ocr(df['full_path'][0])

In [ ]:
df

# upload to ocrProcessing
- /var/www/mnt/shared/TEMP-Patient/{{PATIENT LAST NAME}}{{PATIENT FIRST NAME}}{{RANDOM STRING}}

In [ ]:
df['full_path'] = df['full_path'].map(lambda s: s[s.find('/upds/')+1:])

In [ ]:
df['ocrLogs_id'] = df['id']

In [ ]:
cols = 'id docName num_pages extracted_names best_guess_name full_path ocr_text'.split(' ')
df[cols]

In [ ]:
mycursor = cnxn.cursor()

mycursor.execute("""
describe nerOutput
""")

myresult = mycursor.fetchall()

for x in myresult:
    print(x)

In [ ]:
mycursor = cnxn.cursor()

mycursor.execute("""
describe nerOutput
""")

myresult = mycursor.fetchall()

for x in myresult:
    print(x)

In [ ]:
insert_df = df[[x[0] for x in myresult][1:]]
insert_df['extracted_names'] = insert_df['extracted_names'].astype(str)
insert_df

In [ ]:
# # https://www.dataquest.io/blog/sql-insert-tutorial/
# # creating column list for insertion
# cols = "`,`".join([str(i) for i in insert_df.columns.tolist()])

# # Insert DataFrame recrds one by one.
# for i,row in insert_df.iterrows():
#     sql = "INSERT INTO `nerOutput` (`" +cols + "`) VALUES (" + "%s,"*(len(row)-1) + "%s)"
#     mycursor.execute(sql, tuple(row))

#     # the connection is not autocommitted by default, so we must commit to save our changes
#     connection.commit()

In [ ]:
columns_list = ', '.join(insert_df.columns.to_list())
ses = '%s, ' * (len(insert_df.columns)-1) + '%s'

sql = f"INSERT INTO nerOutput ({columns_list}) VALUES ({ses})"
val = [*map(tuple, insert_df.to_numpy())]

try:
    mycursor.executemany(sql, val)
except Exception as e:
    print(e)

cnxn.commit()

print(mycursor.rowcount, "was inserted.") 

In [ ]:
tuple(insert_df['ocrLogs_id'])

In [ ]:
# # update processed documents
# sql = "UPDATE ocrLogs SET status = 1 WHERE id = %s"
# val = [(x,) for x in insert_df['ocrLogs_id']]

# mycursor.executemany(sql, val)

# cnxn.commit()

# print(mycursor.rowcount, "record(s) affected") 

In [ ]:
# upload files to gcp storage bucket
os.system('gsutil -m cp -r data/pngs/* gs://pngi')
os.system('gsutil -m cp -r data/txts/* gs://txtf')

In [ ]:
# os.system('rm -rf data/pngs/')
# os.system('rm -rf data/txts/')

## png_process.py